In [ ]:
from pyomo.environ import *
import highspy

In [ ]:
Sector = ['Pertanian', 'Manufaktur', 'Jasa', 'Industri']
S = ['A', 'M', 'S', 'I']

In [ ]:
a = {
    'A': {'A': 0.20, 'M': 0.10, 'S': 0.05, 'I': 0.08},
    'M': {'A': 0.05, 'M': 0.25, 'S': 0.10, 'I': 0.12},
    'S': {'A': 0.02, 'M': 0.06, 'S': 0.15, 'I': 0.05},
    'I': {'A': 0.01, 'M': 0.04, 'S': 0.02, 'I': 0.18},
}

In [ ]:
d = {'A': 50, 'M': 75, 'S': 50, 'I': 40}
va = {'A': 0.6, 'M': 0.8, 'S': 0.75, 'I': 0.9}

In [ ]:
labour = {'A': 2, 'M': 3, 'S': 1, 'I': 3}
capital = {'A': 1.1, 'M': 2, 'S': 0.8, 'I': 2.5}
environment = {'A': 0.5, 'M': 0.9, 'S': 0.3, 'I': 1.1}

L_max = 1000
C_max = 1200 
E_max = 400

In [ ]:
m = ConcreteModel()
m.I = Set(initialize=S)
m.p = Var(m.I, domain=NonNegativeReals)

In [ ]:
m.obj = Objective(expr = sum(va[i] * m.p[i] for i in m.I), sense=maximize)

In [ ]:
def io_rule(m, j):
    return sum(a[i][j] * m.p[j] for i in m.I) + d[j] <= m.p[j]
m.io = Constraint(m.I, rule=io_rule)

In [ ]:
m.labour = Constraint(expr = sum(labour[i] * m.p[i] for i in m.I) <= L_max)
m.capital = Constraint(expr = sum(capital[i] * m.p[i] for i in m.I) <= C_max)
m.environment = Constraint(expr = sum(environment[i] * m.p[i] for i in m.I) <= E_max)

In [ ]:
solver = SolverFactory('highs')
results = solver.solve(m, tee=True)

Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 7 rows; 4 cols; 16 nonzeros
Coefficient ranges:
  Matrix  [3e-01, 3e+00]
  Cost    [6e-01, 9e-01]
  Bound   [0e+00, 0e+00]
  RHS     [4e+01, 1e+03]
Presolving model
2 rows, 4 cols, 8 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve reductions: rows 2(-5); columns 3(-1); nonzeros 6(-10) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.2499960003e+00 Ph1: 2(7.9); Du: 3(2.25) 0s
          1    -3.9503588517e+02 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 1
Objective value     :  3.9503588517e+02
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


In [ ]:
print("PDB per Sektor:")
for i, sector in enumerate(m.I):
    print(f"{Sector[i]:12} = {va[sector]*m.p[sector]():>6.2f} ({va[sector]*m.p[sector]()/m.obj()*100:>5.2f}%)")
print(f"TOTAL        = {m.obj():.2f}")


PDB per Sektor:
Pertanian    =  41.67 (10.55%)
Manufaktur   = 109.09 (27.62%)
Jasa         = 181.12 (45.85%)
Industri     =  63.16 (15.99%)
TOTAL        = 395.04


1. Setelah melakukan optimisasi menggunakan solver, 395.04 merupakan nilai PDB tertinggi yang didapatkan berdasarkan ketentuan-ketentuan yang diterapkan
2. Kontributor PDB terbesar adalah Jasa(46%), diikuti oleh Manufaktur(28%). Berikut implikasi yang bisa didapatkan
   - Pemerintah sebaiknya memperkuat sektor jasa (pendidikan, keuangan, pariwisata, digital economy) karena:
     - Efisien dalam penggunaan tenaga kerja dan modal.
     - Memberi kontribusi terbesar terhadap pertumbuhan ekonomi.
   - Manufaktur masih penting untuk kestabilan ekonomi jangka panjang karena efek pengganda ke sektor lain (supply chain, ekspor).
   - Pertanian relatif kecil kontribusinya, tapi tetap vital bagi ketahanan pangan dan kesejahteraan pedesaan.
   - Infrastruktur memberi dampak tidak langsung (meningkatkan efisiensi sektor lain), walau kontribusinya moderat secara langsung.
3. Adanya pembatasan pada tenanga kerja dan modal, tentu berpengaruh besar pada PDB yang akan dihasilkan. Karena adanya pembatasan, maka modal dan tenaga kerja lebih banyak dialokasikan ke dalam sektor yang efisien dalam penggunaan sumber daya, yakni sektor jasa. Jika dilakukan penambahan tenaga kerja dan modal, maka PDM akan semangkin meningkat, dan bisa dilakukan investasi lebih besar pada sektor yang butuh investasi tinggi.